In [1]:
%reset -f
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%config Completer.use_jedi = False

In [2]:
import sys
sys.path.insert(0,"..")
from tqdm import tqdm
# from tqdm.notebook import tqdm as tqdm
import numpy as np
import pandas as pd
import torch
torch.manual_seed(42)
import torch.nn as nn
import torch.optim as optim
import torchaudio
from torch.utils.data import Dataset, ConcatDataset
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import datasets, transforms
torchaudio.set_audio_backend('soundfile')
import os
import random
from pathlib import Path
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import psutil
import requests
import seaborn as sns

from utils import *
from datasets import *
from models import *
from transforms import *
from profiler import *

Available workers: 16


In [3]:
# changed > to >= and < to <= for peaks and min score from 3.5 and maxscore from 12
maxpeaks = 15
minpeaks = 1
maxscore = 15
minscore = 2.5
selection = f"peaks>={minpeaks} & peaks<={maxpeaks} & score>{minscore} & score<={maxscore}"

# Melanogaster and Suzukii main data

In [4]:
dmel1 = WingbeatDatasetProfiler(dsname="Melanogaster_RL/Z", custom_label=[0])
dsuz1 = WingbeatDatasetProfiler(dsname="Suzukii_RL/Y", custom_label=[1])
dmel2 = WingbeatDatasetProfiler(dsname="Melanogaster_RL/Y", custom_label=[0])
dsuz2 = WingbeatDatasetProfiler(dsname="Suzukii_RL/X", custom_label=[1])

Found 24763 in dataset: Melanogaster_RL/Z, and 1 label(s): ['D. melanogaster']
Label(s) changed to [0]


Creating a pandas Dataframe with file-paths, clean-scores, duration, sums of abs values, indice and labels..
Finished.
Found 25732 in dataset: Suzukii_RL/Y, and 1 label(s): ['D. suzukii']
Label(s) changed to [1]


Creating a pandas Dataframe with file-paths, clean-scores, duration, sums of abs values, indice and labels..
Finished.
Found 29002 in dataset: Melanogaster_RL/Y, and 1 label(s): ['D. melanogaster']
Label(s) changed to [0]


Creating a pandas Dataframe with file-paths, clean-scores, duration, sums of abs values, indice and labels..
Finished.
Found 19657 in dataset: Suzukii_RL/X, and 1 label(s): ['D. suzukii']
Label(s) changed to [1]


Creating a pandas Dataframe with file-paths, clean-scores, duration, sums of abs values, indice and labels..
Finished.


In [5]:
# temprs, humds = [],[]
# for i in [dmel1, dmel2, dsuz1, dsuz2]:
#     i.wbts.parse_filenames(temp_humd=True)
#     tmp = i.wbts.df
#     humds.append(tmp.humidity)
#     temprs.append(tmp.temperature)
# humds, temprs = pd.concat(humds), pd.concat(temprs)

# print(f"Average humidity: {(humds[humds>50]-50).mean().round(0)}")
# print(f"std: {(humds[humds>50]-50).std().round(0)}")
# print(f"Average temperature: {temprs.mean().round(1)}")
# print(f"std: {temprs.std().round(1)}")


In [6]:
dfmel1 = dmel1.df.query(selection)
dfmel1.y = 0
dfmel2 = dmel2.df.query(selection)
dfmel2.y = 0

dfsuz1 = dsuz1.df.query(selection)
dfsuz1.y = 1
dfsuz2 = dsuz2.df.query(selection)
dfsuz2.y = 1

/home/kalfasyan/anaconda3/envs/wbai/lib/python3.8/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/home/kalfasyan/anaconda3/envs/wbai/lib/python3.8/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/home/kalfasyan/anaconda3/envs/wbai/lib/python3.8/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

In [7]:
torch.backends.cudnn.benchmark = True
batch_size = 64

In [8]:
train_df = pd.concat([dfmel1, dfsuz1])
test_df = pd.concat([dfmel2, dfsuz2])

print(train_df.y.value_counts())
print(test_df.y.value_counts())

1    19700
0    14599
Name: y, dtype: int64
1    15584
0    14136
Name: y, dtype: int64


# Saving

In [9]:
train_df.to_parquet("../data_created/suzmel_train.parquet")
test_df.to_parquet("../data_created/suzmel_test.parquet")

## MeBioS Suzukii and Melanogaster data

In [10]:
ds1 = WingbeatDatasetProfiler(dsname="Suzukii_RL/R", custom_label=[1])
ds2 = WingbeatDatasetProfiler(dsname="Suzukii_RL/L", custom_label=[1])
ds3 = WingbeatDatasetProfiler(dsname="Melanogaster_RL/X", custom_label=[0])
ds4 = WingbeatDatasetProfiler(dsname="Melanogaster_RL/W", custom_label=[0])

Found 14348 in dataset: Suzukii_RL/R, and 1 label(s): ['D. suzukii']
Label(s) changed to [1]


Creating a pandas Dataframe with file-paths, clean-scores, duration, sums of abs values, indice and labels..
Finished.
Found 21940 in dataset: Suzukii_RL/L, and 1 label(s): ['D. suzukii']
Label(s) changed to [1]


Creating a pandas Dataframe with file-paths, clean-scores, duration, sums of abs values, indice and labels..
Finished.
Found 2086 in dataset: Melanogaster_RL/X, and 1 label(s): ['D. melanogaster']
Label(s) changed to [0]


Creating a pandas Dataframe with file-paths, clean-scores, duration, sums of abs values, indice and labels..
Finished.
Found 1882 in dataset: Melanogaster_RL/W, and 1 label(s): ['D. melanogaster']
Label(s) changed to [0]


Creating a pandas Dataframe with file-paths, clean-scores, duration, sums of abs values, indice and labels..
Finished.


In [11]:
# temprs, humds = [],[]
# for i in [ds1, ds2, ds3, ds4]:
#     i.wbts.parse_filenames(temp_humd=True)
#     tmp = i.wbts.df
#     humds.append(tmp.humidity)
#     temprs.append(tmp.temperature)
# humds, temprs = pd.concat(humds), pd.concat(temprs)

# print(f"Average humidity: {(humds[humds>50]-50).mean().round(0)}")
# print(f"std: {(humds[humds>50]-50).std().round(0)}")
# print(f"Average temperature: {temprs.mean().round(1)}")
# print(f"std: {temprs.std().round(1)}")


In [12]:
extra_df = pd.concat([ds1.df,ds2.df,ds3.df,ds4.df])
extra_df.y.value_counts()

1    36288
0     3968
Name: y, dtype: int64

In [13]:
extra_df = extra_df.query(selection)
extra_df.y.value_counts()

1    24332
0     1382
Name: y, dtype: int64

In [14]:
extra_df.to_parquet("../data_created/suzmel_extra.parquet")

In [15]:
pd.concat([train_df, extra_df]).to_parquet("../data_created/suzmel_traindf_plus_extradf.parquet")

### SD card - novdec2021/jan2022 data

In [16]:
nu1 = WingbeatDatasetProfiler(dsname="/home/kalfasyan/data/wingbeats/Fruitflies_flytrap_rpi/from_sd/Melanogaster/train/", custom_label=[0])
nu2 = WingbeatDatasetProfiler(dsname="/home/kalfasyan/data/wingbeats/Fruitflies_flytrap_rpi/from_sd/Suzukii/", custom_label=[1])
nu_df = pd.concat([nu1.df,nu2.df])

nu_df = nu_df.query(selection)
nu_df.y.value_counts()

Found 11136 in dataset: /home/kalfasyan/data/wingbeats/Fruitflies_flytrap_rpi/from_sd/Melanogaster/train/, and 1 label(s): ['from_sd']
Label(s) changed to [0]


Creating a pandas Dataframe with file-paths, clean-scores, duration, sums of abs values, indice and labels..
Finished.
Found 2725 in dataset: /home/kalfasyan/data/wingbeats/Fruitflies_flytrap_rpi/from_sd/Suzukii/, and 1 label(s): ['from_sd']
Label(s) changed to [1]


Creating a pandas Dataframe with file-paths, clean-scores, duration, sums of abs values, indice and labels..
Finished.


0    6552
1    1951
Name: y, dtype: int64

In [17]:
pd.concat([train_df, extra_df,nu_df]).to_parquet("../data_created/suzmel_traindf_plus_extradf_plus_nudf.parquet")

In [18]:
train_df.y.value_counts()

1    19700
0    14599
Name: y, dtype: int64

In [19]:
pd.concat([train_df, extra_df]).y.value_counts()

1    44032
0    15981
Name: y, dtype: int64

In [20]:
pd.concat([train_df, extra_df,nu_df]).y.value_counts()

1    45983
0    22533
Name: y, dtype: int64

### Rpi - novdevc2021/jan2022 data

In [21]:
rpi1 = WingbeatDatasetProfiler(dsname="/home/kalfasyan/data/wingbeats/Fruitflies_flytrap_rpi/from_rpi/Melanogaster/train/", custom_label=[0], rpiformat=True)
rpi2 = WingbeatDatasetProfiler(dsname="/home/kalfasyan/data/wingbeats/Fruitflies_flytrap_rpi/from_rpi/Suzukii/train/", custom_label=[1], rpiformat=True)
rpi_df = pd.concat([rpi1.df,rpi2.df])

rpi_df = rpi_df.query(selection)
rpi_df.y.value_counts()

Found 216 in dataset: /home/kalfasyan/data/wingbeats/Fruitflies_flytrap_rpi/from_rpi/Melanogaster/train/, and 1 label(s): ['from_rpi']
Label(s) changed to [0]


Creating a pandas Dataframe with file-paths, clean-scores, duration, sums of abs values, indice and labels..
Finished.
Found 0 in dataset: /home/kalfasyan/data/wingbeats/Fruitflies_flytrap_rpi/from_rpi/Suzukii/train/, and 0 label(s): []
Label(s) changed to [1]


Creating a pandas Dataframe with file-paths, clean-scores, duration, sums of abs values, indice and labels..
Finished.


0.0    206
Name: y, dtype: int64

In [24]:
pd.concat([train_df, nu_df, rpi_df]).to_parquet("../data_created/suzmel_traindf_plus_nudf_plus_rpidf.parquet")#.y.value_counts()

In [25]:
pd.concat([train_df, extra_df, rpi_df]).to_parquet("../data_created/suzmel_traindf_plus_extra_plus_rpidf.parquet")#.y.value_counts()

In [26]:
pd.concat([train_df, extra_df,  nu_df, rpi_df]).to_parquet("../data_created/suzmel_traindf_plus_extra_plus_nudf_plus_rpidf.parquet")#.y.value_counts()

In [114]:
extra_df.shape

(22744, 13)